In [1]:
import pandas as pd

In [2]:
loans = pd.DataFrame.from_csv('lending-club-data.csv')

D:\Users\zeyang\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2825: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: 1 if x == 0 else -1)
del loans['bad_loans']

In [4]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [6]:
loans = loans[features+[target]]

In [8]:
loans = pd.get_dummies(loans)

In [39]:
train_data = loans.iloc[pd.read_json('module-6-assignment-train-idx.json')[0]].reset_index(drop=True)
validation_data = loans.iloc[pd.read_json('module-6-assignment-validation-idx.json')[0]].reset_index(drop=True)

In [40]:
def reached_minimum_node_size(data, min_node_size):
    return len(data) < min_node_size

In [41]:
# Quiz question: Given an intermediate node with 6 safe loans and 3 risky loans, 
# if the min_node_size parameter is 10, what should the tree learning algorithm do next?

# Stop

In [42]:
def error_reduction(error_before_split, error_after_split):
    return error_before_split - error_after_split

In [43]:
# Quiz question: Assume an intermediate node has 6 safe loans and 3 risky loans. 
# For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, 
# and 0.14, respectively. If the minimum gain in error reduction parameter is set to 0.2, 
# what should the tree learning algorithm do next?

# Stop

In [103]:
def intermediate_node_num_mistakes(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    num_ones = len(labels_in_node[labels_in_node == 1])
    num_zeros = len(labels_in_node[labels_in_node == -1])
    return num_ones if num_zeros >= num_ones else num_zeros

In [104]:
def best_splitting_feature(data, features, target):
    target_values = data[target]
    best_feature = None
    best_error = 10
    
    num_data_points = float(len(data))
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        left_mistake = intermediate_node_num_mistakes(left_split[target])
        right_mistake = intermediate_node_num_mistakes(right_split[target])
        error = (left_mistake + right_mistake) / num_data_points
        if error < best_error:
            best_error = error
            best_feature = feature
    
    print('Best feature found {} error {}'.format(best_feature, best_error))
    return best_feature

In [105]:
class Node:
    def __init__(self, splitting_feature=None, is_leaf=False, left=None, right=None, prediction=None):
        self.splitting_feature = splitting_feature
        self.is_leaf = is_leaf
        self.left = left
        self.right = right
        self.prediction = prediction

In [106]:
def create_leaf(target_values):
    leaf = Node(is_leaf=True)
    num_pos = len(target_values[target_values == 1])
    num_neg = len(target_values[target_values == -1])
    if num_pos >= num_neg:
        leaf.prediction = 1
    else:
        leaf.prediction = -1
    return leaf

In [107]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    remaining_features = features[:]
    
    # Stop condition
    if intermediate_node_num_mistakes(data[target]) == 0:
        print('Reached stopping condition - all same category')
        return create_leaf(data[target])
    if remaining_features == []:
        print('Reached stopping condition - empty features')
        return create_leaf(data[target])
    # Early stopping condition 1
    if current_depth >= max_depth:
        print('Reached early stopping condition - reached max depth')
        return create_leaf(data[target])
    # Early stopping condition 2
    if reached_minimum_node_size(data, min_node_size):
        print('Reached early stopping condition - too few data in this node')
        return create_leaf(data[target])
    # Early sopping condition 3
    
    splitting_feature = best_splitting_feature(data, remaining_features, target)
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    error_before_split = intermediate_node_num_mistakes(data[target]) / float(len(data))
    left_mistake = intermediate_node_num_mistakes(left_split[target])
    right_mistake = intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistake + right_mistake) / float(len(data))
    
    if error_reduction(error_before_split, error_after_split) < min_error_reduction:
        print('Reached early stopping condition - error reduction is less than min_error_reduction')
        return create_leaf(data[target])
    
    remaining_features.remove(splitting_feature)
    if len(left_split) == len(data):
        print('Creating leaf node')
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print('Creating leaf node')
        return create_leaf(right_split[target])
    
    print('Split on feature {}, ({}, {})'.format(splitting_feature, len(left_split), len(right_split)))

    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth+1, 
                                     max_depth, min_node_size, min_error_reduction)
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth+1,
                                     max_depth, min_node_size, min_error_reduction)
    return Node(splitting_feature=splitting_feature, left=left_tree, right=right_tree)

In [108]:
features = loans.columns.values.tolist()
features.remove('safe_loans')

In [109]:
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                            min_node_size = 100, min_error_reduction=0.0)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found grade_B error 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found grade_C error 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found grade_D error 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found grade_E error 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth
Best feature found grade_E error 0.35131195335276966
Creating leaf node
Best feature found grade_D error 0.41735159817351597
Creating leaf node
Best feature found emp_length_5 years error 0.43702290076335876
Split on feature emp_length_5 years, (969, 79)
Best feature found grade_C error 0.43343653250773995
Creating leaf node
Reac

In [110]:
my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, min_node_size = 0, min_error_reduction=-1)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found grade_B error 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found grade_C error 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found grade_D error 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found grade_E error 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth
Best feature found grade_E error 0.35131195335276966
Creating leaf node
Best feature found grade_D error 0.41735159817351597
Creating leaf node
Best feature found emp_length_5 years error 0.43702290076335876
Split on feature emp_length_5 years, (969, 79)
Best feature found grade_C error 0.43343653250773995
Creating leaf node
Best

In [111]:
def classify(tree, x, annotate=False):
    if tree.is_leaf:
        if annotate:
            print('At leaf, feature {} predicting {}'.format(tree.splitting_feature,tree.prediction))
        return tree.prediction
    if annotate:
        print('Split on {} = {}'.format(tree.splitting_feature, x[tree.splitting_feature]))
    if x[tree.splitting_feature] == 1:
        return classify(tree.right, x, annotate)
    return classify(tree.left, x, annotate)

In [112]:
print(validation_data.ix[0])

safe_loans                -1
grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            0
term_ 60 months            1
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         1
emp_length_3 years         0
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
emp_length_n/a             0
Name: 0, dtype: float64


In [113]:
print('predicted class: {}'.format(classify(my_decision_tree_new, validation_data.ix[0])))

predicted class: -1


In [114]:
print('predicted class: {}'.format(classify(my_decision_tree_new, validation_data.ix[0], True)))

Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
At leaf, feature None predicting -1
predicted class: -1


In [115]:
print('predicted class: {}'.format(classify(my_decision_tree_old, validation_data.ix[0], True)))

Split on term_ 36 months = 0.0
Split on grade_A = 0.0
Split on grade_B = 0.0
Split on grade_C = 0.0
Split on grade_D = 1.0
At leaf, feature None predicting -1
predicted class: -1


In [116]:
def evaluate_classification_error(tree, data, target):
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    return sum(prediction == data[target]) / len(data)

In [117]:
evaluate_classification_error(my_decision_tree_new, validation_data, target)

0.61589831968978892

In [118]:
evaluate_classification_error(my_decision_tree_old, validation_data, target)

0.61622145626884961

In [119]:
model_1 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                               min_node_size = 0, min_error_reduction=-1)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth
Best feature found grade_D error 0.4175922288489697
Split on feature grade_D, (23300, 4701)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth


In [120]:
model_2 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 0, min_error_reduction=-1)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found grade_B error 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found grade_C error 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found grade_D error 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found grade_E error 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth
Best feature found grade_E error 0.35131195335276966
Creating leaf node
Best feature found grade_D error 0.41735159817351597
Creating leaf node
Best feature found emp_length_5 years error 0.43702290076335876
Split on feature emp_length_5 years, (969, 79)
Best feature found grade_C error 0.43343653250773995
Creating leaf node
Best

In [121]:
model_3 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 14, 
                               min_node_size = 0, min_error_reduction=-1)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found grade_B error 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found grade_C error 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found grade_D error 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found grade_E error 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Best feature found home_ownership_OTHER error 0.2575310100413467
Split on feature home_ownership_OTHER, (1692, 1)
Best feature found grade_F error 0.2576832151300236
Split on feature grade_F, (339, 1353)
Best feature found grade_G error 0.2448377581120944
Creating leaf node
Best feature found grade_G error 0.2609016999260902
Creating leaf node
Reached stopping condition - all same category
Best feature found grade_F error 0.292545710

In [122]:
print("Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data, target))
print("Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data, target))
print("Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data, target))

Training data, classification error (model 1): 0.599962389856
Training data, classification error (model 2): 0.618149580916
Training data, classification error (model 3): 0.623817966903


In [123]:
print("Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_data, target))
print("Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_data, target))
print("Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_data, target))

Validation data, classification error (model 1): 0.601895734597
Validation data, classification error (model 2): 0.616221456269
Validation data, classification error (model 3): 0.62268418785


In [124]:
def count_leaves(tree):
    if tree.is_leaf:
        return 1
    return count_leaves(tree.left) + count_leaves(tree.right)

In [125]:
count_leaves(model_1)

4

In [126]:
count_leaves(model_2)

19

In [127]:
count_leaves(model_3) 

41

In [128]:
# Explor the effect of min_error_reduction

In [129]:
model_4 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 0, min_error_reduction=-1)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found grade_B error 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found grade_C error 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found grade_D error 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found grade_E error 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth
Best feature found grade_E error 0.35131195335276966
Creating leaf node
Best feature found grade_D error 0.41735159817351597
Creating leaf node
Best feature found emp_length_5 years error 0.43702290076335876
Split on feature emp_length_5 years, (969, 79)
Best feature found grade_C error 0.43343653250773995
Creating leaf node
Best

In [130]:
model_5 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 0, min_error_reduction=0)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found grade_B error 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found grade_C error 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found grade_D error 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found grade_E error 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth
Best feature found grade_E error 0.35131195335276966
Creating leaf node
Best feature found grade_D error 0.41735159817351597
Creating leaf node
Best feature found emp_length_5 years error 0.43702290076335876
Split on feature emp_length_5 years, (969, 79)
Best feature found grade_C error 0.43343653250773995
Creating leaf node
Best

In [131]:
model_6 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 0, min_error_reduction=5)

Best feature found term_ 36 months error 0.4216365785514722
Reached early stopping condition - error reduction is less than min_error_reduction


In [132]:
print("Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data, target))
print("Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data, target))
print("Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data, target))

Validation data, classification error (model 4): 0.616221456269
Validation data, classification error (model 5): 0.616221456269
Validation data, classification error (model 6): 0.496553209823


In [133]:
count_leaves(model_4)

19

In [134]:
count_leaves(model_5)

19

In [135]:
count_leaves(model_6)

1

In [136]:
# Explor the effect of min_node_size

In [137]:
model_7 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 0, min_error_reduction=-1)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found grade_B error 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found grade_C error 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found grade_D error 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found grade_E error 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth
Best feature found grade_E error 0.35131195335276966
Creating leaf node
Best feature found grade_D error 0.41735159817351597
Creating leaf node
Best feature found emp_length_5 years error 0.43702290076335876
Split on feature emp_length_5 years, (969, 79)
Best feature found grade_C error 0.43343653250773995
Creating leaf node
Best

In [138]:
model_8 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 2000, min_error_reduction=-1)

Best feature found term_ 36 months error 0.4216365785514722
Split on feature term_ 36 months, (9223, 28001)
Best feature found grade_A error 0.34674184104955
Split on feature grade_A, (9122, 101)
Best feature found grade_B error 0.34630563472922604
Split on feature grade_B, (8074, 1048)
Best feature found grade_C error 0.33415902898191724
Split on feature grade_C, (5884, 2190)
Best feature found grade_D error 0.30319510537049627
Split on feature grade_D, (3826, 2058)
Best feature found grade_E error 0.2773131207527444
Split on feature grade_E, (1693, 2133)
Reached early stopping condition - reached max depth
Reached early stopping condition - reached max depth
Best feature found grade_E error 0.35131195335276966
Creating leaf node
Best feature found grade_D error 0.41735159817351597
Creating leaf node
Reached early stopping condition - too few data in this node
Reached early stopping condition - too few data in this node
Best feature found grade_D error 0.4175922288489697
Split on feat

In [89]:
model_9 = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 50000, min_error_reduction=-1)

Reached early stopping condition - too few data in this node


In [91]:
print("Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_data, target))
print("Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_data, target))
print("Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_data, target))

Validation data, classification error (model 7): 0.616221456269
Validation data, classification error (model 8): 0.615467470918
Validation data, classification error (model 9): 0.496553209823


In [92]:
count_leaves(model_7)

19

In [93]:
count_leaves(model_8)

12

In [94]:
count_leaves(model_9)

1